In [1]:
import pandas as pd
from fastai.text.all import *

In [2]:
path = untar_data(URLs.IMDB)

In [3]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

In [4]:
 imdb_dls = DataBlock(blocks=TextBlock.from_folder(path, is_lm=True),
                       get_items=get_imdb, splitter=RandomSplitter(0.2)).dataloaders(path, path=path, bs=128, seq_len=80)

In [5]:
dls_clas = DataBlock(
                     blocks=(TextBlock.from_folder(path, vocab=imdb_dls.vocab),CategoryBlock),
                     get_y = parent_label,
                     get_items=partial(get_text_files, folders=['train', 'test']),
                     splitter=GrandparentSplitter(valid_name='test')
                     ).dataloaders(path, path=path, bs=128, seq_len=72)

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck


In [6]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy).to_fp16()

C:\Users\user\miniconda3\envs\DA_ENV\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [7]:
learn.load("imdb_inferer")

In [8]:
review = "The movie was great."

In [9]:
learn.predict(review)

C:\Users\user\miniconda3\envs\DA_ENV\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


('pos', TensorText(1), TensorText([0.0022, 0.9978]))

In [21]:
df = pd.read_csv("Data/imdb_newsamples.csv")


In [22]:
reviews = df["review"]

In [23]:
review = reviews[2]

In [24]:
type(review)

str

In [25]:
learn.predict(review)

C:\Users\user\miniconda3\envs\DA_ENV\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


('neg', TensorText(0), TensorText([0.9659, 0.0341]))